In [9]:
import ollama
import gradio as gr
import json

import nbimporter

from events_api import get_events_info_from_city
from city_corrections import get_city

In [10]:
MODEL = "llama3.2"

In [11]:
system_prompt = "You are EventAIde, an efficient AI assistant that helps users find events in the city the user mentioned. "
system_prompt += "At the start of every reply, introduce yourself as 'EventAIde'. "
system_prompt += "Always ask user for the city and if the user doesnt mention then give any city's events. "
system_prompt += "Give only the requested event details in a short, friendly, and professional reply. "
system_prompt += "If you do not KNOW the requested event details for a city, simply reply:\
'Sorry, I do not have information for that city at this time.' \
Never make up any details or invent events."

system_message = [{"role":"system", "content": system_prompt}]

In [12]:
event_function = {
    "name": "get_events_info_from_city",
    "description": "Get the events happening from the given city. Call this whenever you need to know the events list, for example when a customer asks ‘what events are there in New York'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "All the available events in the customer provided city",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": event_function}]

In [13]:
events_data_ex = {
    "Sports": {
        "Baseball": [
            {
                "name": "A vs. B",
                "date": "2025-08-16",
                "time": "18:15:00",
                "venue": {
                    "name": "ABC Stadium",
                    "address": "420 S Street.",
                    "city": "ABC city",
                    "state": "DEF"
                },
                "subgenre": "MLB"
            },
            # More baseball events...
        ]
        # More sports genres...
    },
    "Music": {
        "Rock": [
            {
                "name": "The ABC Band",
                "date": "2025-08-17",
                "time": "19:30:00",
                "venue": {
                    "name": "The Factory",
                    "address": "1710 N Road",
                    "city": "ABC city",
                    "state": "DEF"
                },
                "subgenre": "Rock"
            },
            # More rock events...
        ]
        # More music genres...
    },
    "Arts & Theatre": {
        "Comedy": [
            {
                "name": "Show",
                "date": "2025-10-22",
                "time": "20:00:00",
                "venue": {
                    "name": "The ABC",
                    "address": "6161 Delmar",
                    "city": "ABC City",
                    "state": "DEF"
                },
                "subgenre": "Comedy"
            },
            # More comedy events...
        ],
        # More arts genres...
    },
    # More top-level categories (e.g., 'Miscellaneous', 'Film') as needed...
}


system_promp_city = f"""
You have a variable 'events_data' that looks like this:
{repr(events_data_ex)}

When both city and type are known, output the list in this format (one per line and grouped by category):
Sports:
event name - venue name - date
Music:
event name - venue name - date

For example:
Sports:
A vs. B - ABC Stadium - 2025-08-16
Music:
The ABC Band - The Factory - 2025-08-17
"""

In [14]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = tool_call.function.arguments
    city = get_city(arguments.get('city'))
    events = get_events_info_from_city(city)
    response = {
        "role": "tool",
        "content": json.dumps({"city": city,"events": events})
    }
    return response, city

In [15]:
def chatbot(message, history):
    messages = system_message+history+[{"role":"user", "content":message}]
    response = ollama.chat(model = MODEL, messages=messages, tools=tools)
    if response.message.tool_calls:
        message = response.message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append({"role":"system", "content":system_promp_city})
        messages.append(response)
        response = ollama.chat(model=MODEL, messages=messages)
        
    return response["message"]["content"]

In [16]:
gr.ChatInterface(fn = chatbot, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
